In [1]:
from torch._export import capture_pre_autograd_graph

In [2]:
import torch
from torch import nn
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(1, 1, 3, 3)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv(x)
        x = self.relu(x)
        return x

In [3]:
net = Net()
x  = torch.randn(1, 1, 10, 10)
exported = capture_pre_autograd_graph(net, (x, ))

In [4]:
print(exported)

GraphModule()



def forward(self, x):
    arg0, = fx_pytree.tree_flatten_spec(([x], {}), self._in_spec)
    _param_constant0 = self._param_constant0
    _param_constant1 = self._param_constant1
    conv2d_default = torch.ops.aten.conv2d.default(arg0, _param_constant0, _param_constant1, [3, 3]);  arg0 = _param_constant0 = _param_constant1 = None
    relu_default = torch.ops.aten.relu.default(conv2d_default);  conv2d_default = None
    return pytree.tree_unflatten([relu_default], self._out_spec)
    
# To see more debug info, please use `graph_module.print_readable()`


In [5]:
from pprint import pprint
for n in exported.graph.nodes:
    print("-----------------")
    pprint(n.meta)

-----------------
{'is_torch_exported': True, 'val': FakeTensor(..., size=(1, 1, 10, 10))}
-----------------
{'from_node': [('l__self___conv', 'L__self___conv'),
               ('_param_constant0', '_param_constant0')],
 'is_torch_exported': True,
 'nn_module_stack': {'L__self___conv': ("L['self'].conv",
                                        <class 'torch.nn.modules.conv.Conv2d'>)},
 'original_aten': <OpOverload(op='aten.conv2d', overload='default')>,
 'seq_nr': 5,
 'source_fn': ('l__self___conv', <class 'torch.nn.modules.conv.Conv2d'>),
 'stack_trace': '  File "/tmp/ipykernel_163612/1464477655.py", line 10, in '
                'forward\n'
                '    x = self.conv(x)\n'}
-----------------
{'from_node': [('l__self___conv', 'L__self___conv'),
               ('_param_constant1', '_param_constant1')],
 'is_torch_exported': True,
 'nn_module_stack': {'L__self___conv': ("L['self'].conv",
                                        <class 'torch.nn.modules.conv.Conv2d'>)},
 'original

In [6]:
list(exported.named_modules())

[('', GraphModule())]

In [7]:
exported.print_readable()

class GraphModule(torch.nn.Module):
    def forward(self, x):
        arg0: f32[1, 1, 10, 10], = fx_pytree.tree_flatten_spec(([x], {}), self._in_spec)
        # File: /tmp/ipykernel_163612/1464477655.py:10, code: x = self.conv(x)
        _param_constant0 = self._param_constant0
        _param_constant1 = self._param_constant1
        conv2d_default: f32[1, 1, 3, 3] = torch.ops.aten.conv2d.default(arg0, _param_constant0, _param_constant1, [3, 3]);  arg0 = _param_constant0 = _param_constant1 = None
        
        # File: /tmp/ipykernel_163612/1464477655.py:11, code: x = self.relu(x)
        relu_default: f32[1, 1, 3, 3] = torch.ops.aten.relu.default(conv2d_default);  conv2d_default = None
        return pytree.tree_unflatten([relu_default], self._out_spec)
        


'class GraphModule(torch.nn.Module):\n    def forward(self, x):\n        arg0: f32[1, 1, 10, 10], = fx_pytree.tree_flatten_spec(([x], {}), self._in_spec)\n        # File: /tmp/ipykernel_163612/1464477655.py:10, code: x = self.conv(x)\n        _param_constant0 = self._param_constant0\n        _param_constant1 = self._param_constant1\n        conv2d_default: f32[1, 1, 3, 3] = torch.ops.aten.conv2d.default(arg0, _param_constant0, _param_constant1, [3, 3]);  arg0 = _param_constant0 = _param_constant1 = None\n        \n        # File: /tmp/ipykernel_163612/1464477655.py:11, code: x = self.relu(x)\n        relu_default: f32[1, 1, 3, 3] = torch.ops.aten.relu.default(conv2d_default);  conv2d_default = None\n        return pytree.tree_unflatten([relu_default], self._out_spec)\n        '

In [8]:
from torch.ao.quantization.pt2e.utils import _get_node_name_to_scope
x = _get_node_name_to_scope(exported)

In [9]:
x


{'arg0': ('', NoneType),
 '_param_constant0': ('conv', torch.nn.modules.conv.Conv2d),
 '_param_constant1': ('conv', torch.nn.modules.conv.Conv2d),
 'conv2d_default': ('conv', torch.nn.modules.conv.Conv2d),
 'relu_default': ('relu', torch.nn.modules.activation.ReLU),
 'output': ('', NoneType)}

In [18]:
from torch.ao.quantization.quantize_pt2e import prepare_qat_pt2e
from torch.ao.quantization.quantizer.xnnpack_quantizer import (
    get_symmetric_quantization_config,
    XNNPACKQuantizer,
)
quantizer = XNNPACKQuantizer()
quantizer.set_global(
            get_symmetric_quantization_config(
                is_per_channel=False, is_qat=True
            )
        )
example_inputs = (torch.randn(1, 1, 10, 10),)
model_pt2e = capture_pre_autograd_graph(
            net,
            example_inputs,
        )
model_pt2e = prepare_qat_pt2e(model_pt2e, quantizer)

In [19]:
model_pt2e

GraphModule(
  (activation_post_process_0): FusedMovingAvgObsFakeQuantize(
    fake_quant_enabled=tensor([1]), observer_enabled=tensor([1]), scale=tensor([1.]), zero_point=tensor([0], dtype=torch.int32), dtype=torch.qint8, quant_min=-128, quant_max=127, qscheme=torch.per_tensor_affine, reduce_range=False
    (activation_post_process): MovingAverageMinMaxObserver(min_val=inf, max_val=-inf)
  )
  (activation_post_process_1): FusedMovingAvgObsFakeQuantize(
    fake_quant_enabled=tensor([1]), observer_enabled=tensor([1]), scale=tensor([1.]), zero_point=tensor([0], dtype=torch.int32), dtype=torch.qint8, quant_min=-127, quant_max=127, qscheme=torch.per_tensor_symmetric, reduce_range=False
    (activation_post_process): MovingAverageMinMaxObserver(min_val=inf, max_val=-inf)
  )
  (activation_post_process_2): FusedMovingAvgObsFakeQuantize(
    fake_quant_enabled=tensor([1]), observer_enabled=tensor([1]), scale=tensor([1.]), zero_point=tensor([0], dtype=torch.int32), dtype=torch.qint8, quant_mi

In [20]:
net

Net(
  (conv): Conv2d(1, 1, kernel_size=(3, 3), stride=(3, 3))
  (relu): ReLU()
)

In [23]:
model_pt2e(example_inputs[0])

tensor([[[[0.0000, 0.4099, 0.0916],
          [0.0756, 0.1704, 0.2025],
          [0.0916, 0.1399, 0.0000]]]],
       grad_fn=<FusedMovingAvgObsFqHelperBackward0>)

In [26]:
print(model_pt2e.print_readable())

class GraphModule(torch.nn.Module):
    def forward(self, x):
        arg0: f32[1, 1, 10, 10], = fx_pytree.tree_flatten_spec(([x], {}), self._in_spec)
        # No stacktrace found for following nodes
        activation_post_process_0 = self.activation_post_process_0(arg0);  arg0 = None
        
        # File: /tmp/ipykernel_163612/1464477655.py:10, code: x = self.conv(x)
        _param_constant0 = self._param_constant0
        
        # No stacktrace found for following nodes
        activation_post_process_1 = self.activation_post_process_1(_param_constant0);  _param_constant0 = None
        
        # File: /tmp/ipykernel_163612/1464477655.py:10, code: x = self.conv(x)
        _param_constant1 = self._param_constant1
        conv2d_default: f32[1, 1, 3, 3] = torch.ops.aten.conv2d.default(activation_post_process_0, activation_post_process_1, _param_constant1, [3, 3]);  activation_post_process_0 = activation_post_process_1 = _param_constant1 = None
        
        # File: /tmp/ipyk

AttributeError: 'GraphModule' object has no attribute 'conv'